In [1]:
from JKBot_Class import GameHooker

import pymem
import pymem.pattern
import re
import struct
import time
import numpy as np
import subprocess
import os
import win32gui
import win32con
import win32api
import threading

def to_pattern(hex_string):
    """Zamienia '8D 41 24 83 C0 18' na b'\x8d\x41\x24\x83\xc0\x18'"""
    # Usuwamy spacje i myślniki, jeśli istnieją
    clean_hex = hex_string.replace(" ", "").replace("-", "")
    return bytes.fromhex(clean_hex)

def monitor_loop(hooker):
    """
    Ten wątek działa asynchronicznie, najszybciej jak to możliwe.
    Jego jedynym zadaniem jest aktualizacja stanu i wyświetlanie logów.
    """
    while True:
        try:
            # 1. Szybki odczyt z pamięci
            skoki = hooker.read_value('skoki')
            
            # Bezpieczny odczyt X/Y
            pos_base = hooker.get_adr("pozycja_x")
            if pos_base != 0:
                x = hooker.read_memory(pos_base, "float") - 8
                y = -hooker.read_memory(pos_base + 0x04, "float") + 302.0
            else:
                x, y = 0.0, 0.0

            # Bezpieczny odczyt kierunku
            d_adr = hooker.get_adr("kierunek_raw")
            if d_adr != 0:
                kierunek = hooker.read_memory(d_adr + 0x24)
            else:
                kierunek = 0
                
            lvl = hooker.read_value("lvl", "int_stat")

            # 2. Zapis do globalnego stanu (aby bot mógł z tego korzystać)
            game_state["skoki"] = skoki
            game_state["x"] = x
            game_state["y"] = y
            game_state["kierunek"] = kierunek
            game_state["lvl"] = lvl

            # 3. Drukowanie w jednej linii (często odświeżane)
            kierunek_str = 'LEWO[1]' if kierunek else 'PRAWO[0]'
            print(f"skoki: {skoki} | pozycja_x: {round(x,2)} | pozycja_y: {round(y,2)} | "
                  f"kierunek: {kierunek_str} | lvl: {lvl}                 ", end="\r")
            
            # Taktowanie monitora: np. 100 razy na sekundę (0.01)
            time.sleep(0.05) 
            
        except Exception as e:
            # Ignoruj chwilowe błędy odczytu (np. podczas loadingu mapy)
            pass

In [4]:
# 1. Ścieżka do Twojej gry (podaj pełną ścieżkę do pliku .exe)
game_path = r"C:\GOG Games\Jump King"
game_name = "JumpKing"
free_hand = False

game_state = {
    "skoki": 0,
    "x": 0.0,
    "y": 0.0,
    "kierunek": 0,
    "lvl": 0
}

try:
    # Uruchomienie procesu gry
    print(f"Uruchamiam grę: {game_name}")
    game = subprocess.Popen(os.path.join(game_path, game_name+".exe"), cwd=game_path)
   
    # 2. Czekaj 15 sekund na załadowanie menu/silnika
    print("Czekam 12 sekund...")
    time.sleep(12.0)
    
    hwnd = win32gui.FindWindow(None, "Jump King")
    hooker = GameHooker("JumpKing.exe")


    #PRELOAD HOOKS
    # Dodajemy tyle hooków, ile chcemy
    hooker.add_static_address("lvl",to_pattern("C2 10 00 89 3D"),point=5)

    #JESLI ZMIANA W CORE TO MONO[INFO] - Keyboard ->keyState adress (x2E to wildcard binarny)
    hooker.add_pure_field("key_state_array", re.compile(b'\xE8\xDC\x2E\x70[\x00\x01]{12}', re.DOTALL), offset=0x08,protection=0x04)
    
    #TAKE CONTROL CORE INFECT
    if not free_hand:
        hooker.setup_absolute_bot_control()

    win32gui.SendMessage(hwnd, win32con.WM_ACTIVATE, 0, 0)
    win32gui.SendMessage(hwnd, win32con.WM_KILLFOCUS, 0, 0)

    hooker.unpin_window()
    hooker.fix_fps_drop()

    win32gui.SendMessage(hwnd, win32con.WM_ACTIVATE, 1, 0)
    win32gui.SendMessage(hwnd, win32con.WM_SETFOCUS, 0, 0)

    #raise
    time.sleep(2.0)
    # SPACJA ENTER POCZEKAJ SPACJA ZAMKNIJ
    hooker.input_move(["SPACEBAR"], duration=0.5)
    time.sleep(2.0)
    hooker.input_move(["ENTER"], duration=1.0)
    time.sleep(8.0)
    hooker.input_move(["SPACEBAR"], duration=1.5)
    
    #IN-GAME HOOK Load
    hooker.add_hook("skoki", to_pattern("81 7E 28 E8 03 00 00"), "esi", 0x28) #8.0-454.0
    hooker.add_hook("pozycja_x", to_pattern("D9 1B 8D 5E 2C"), "ebx", 0x00)
    hooker.hooks["pozycja_y"] = {"mailbox":hooker.hooks["pozycja_x"]["mailbox"],"offset":0x04}
    hooker.add_hook("kierunek_raw", to_pattern("85 D2 75 02 5D C3 85 D2"), "ecx", 0x0) #0x10

    # Uruchamiamy monitor w tle (daemon=True oznacza, że wątek umrze razem z głównym programem)
    monitor_thread = threading.Thread(target=monitor_loop, args=(hooker,), daemon=True)
    monitor_thread.start()

    # Dajemy mu ułamek sekundy na pierwsze zasilenie słownika game_state
    time.sleep(0.1)
    
    time.sleep(2.0)
    
    #BOT LOOP
    try:
        while True:            
            # Wszystkie odczyty następują jeden po drugim najszybciej jak się da
            #skoki = hooker.read_value('skoki')
            #x = hooker.read_value('pozycja_x',"float") -8 #"float"
            #y = -hooker.read_value('pozycja_y',"float") +302.0 #"float"
            
            #d_adr = hooker.get_adr("kierunek_raw")
            #if d_adr != 0:
             #   kierunek = hooker.read_memory(d_adr + 0x24)
            #else:
             #   kierunek = 0 # Wartość domyślna, dopóki gra nie zainicjuje adresu
            #kierunek = hooker.read_memory(hooker.get_adr("kierunek_raw")+0x24)
            #lvl = hooker.read_value("lvl","int_stat")
            
            #print(f"skoki: {skoki} | ", end="")
            #print(f"pozycja_x: {round(x,2)} | ", end="")
            #print(f"pozycja_y: {round(y,2)} | ", end="")
            #print(f"kierunek_raw: {('LEWO[1]' if kierunek else 'PRAWO[0]')} | ", end="")
            #print(f"screen_lp: {lvl} | ", end="")
            #print("                 ",end="")
            #print(end="\r")

            hooker.input_move([np.random.choice(["LEFT","RIGHT","PASS"])],duration=0.5*np.random.random())
            hooker.input_move(["SPACEBAR",np.random.choice(["LEFT","RIGHT"])], duration=np.random.random())
            time.sleep(0.01)

    except KeyboardInterrupt:
        print("\nZatrzymano.")
    
    print("Zadanie zakończone.")
    game.kill()

except Exception as e:
    print(f"Wystąpił błąd: {e}")
    #game.kill()

Uruchamiam grę: JumpKing
Czekam 12 sekund...
 New Hook [S]: [0x2ee5978] lvl
 New Pure Field: [0x37b8f00] key_state_array
[*] Inicjalizacja Pełnego Przejęcia XNA/MonoGame...
 [V] Sfałszowana Tablica zaalokowana pod: 0x17970000
 [*] Szukanie OnDeactivate (AOB Scan)...
 [V] Okno odpięte (OnDeactivate spatchowane) pod: 0x17960648
 [*] Szukanie blokady FPS w Game::Tick...
 [V] FPS Fix zaaplikowany pod: 0xe48993d (JNE -> JMP)
 New Hook: [~0x1798ce66] skoki
 New Hook: [~0x1798ad84] pozycja_x
 New Hook: [~0x179860ab] kierunek_raw
skoki: 3662 | pozycja_x: 176.0 | pozycja_y: 54.8 | kierunek: LEWO[1] | lvl: 0                          
Zatrzymano.
Zadanie zakończone.
skoki: 3662 | pozycja_x: 172.5 | pozycja_y: 56.79 | kierunek: LEWO[1] | lvl: 0                 

In [6]:
import threading
import time
import numpy as np

# Współdzielony stan gry - bot będzie na jego podstawie podejmował decyzje
game_state = {
    "skoki": 0,
    "x": 0.0,
    "y": 0.0,
    "kierunek": 0,
    "lvl": 0
}

def monitor_loop(hooker):
    """
    Ten wątek działa asynchronicznie, najszybciej jak to możliwe.
    Jego jedynym zadaniem jest aktualizacja stanu i wyświetlanie logów.
    """
    while True:
        try:
            # 1. Szybki odczyt z pamięci
            skoki = hooker.read_value('skoki')
            
            # Bezpieczny odczyt X/Y
            pos_base = hooker.get_adr("pozycja_x")
            if pos_base != 0:
                x = hooker.read_memory(pos_base, "float") - 8
                y = -hooker.read_memory(pos_base + 0x04, "float") + 302.0
            else:
                x, y = 0.0, 0.0

            # Bezpieczny odczyt kierunku
            d_adr = hooker.get_adr("kierunek_raw")
            if d_adr != 0:
                kierunek = hooker.read_memory(d_adr + 0x24)
            else:
                kierunek = 0
                
            lvl = hooker.read_value("lvl", "int_stat")

            # 2. Zapis do globalnego stanu (aby bot mógł z tego korzystać)
            game_state["skoki"] = skoki
            game_state["x"] = x
            game_state["y"] = y
            game_state["kierunek"] = kierunek
            game_state["lvl"] = lvl

            # 3. Drukowanie w jednej linii (często odświeżane)
            kierunek_str = 'LEWO[1]' if kierunek else 'PRAWO[0]'
            print(f"skoki: {skoki} | pozycja_x: {round(x,2)} | pozycja_y: {round(y,2)} | "
                  f"kierunek: {kierunek_str} | lvl: {lvl}                 ", end="\r")
            
            # Taktowanie monitora: np. 100 razy na sekundę (0.01)
            time.sleep(0.01) 
            
        except Exception as e:
            # Ignoruj chwilowe błędy odczytu (np. podczas loadingu mapy)
            pass

b'\x00\x00\x10B'
b'f3CA'
b'\x00\x00\x10B'
b'f3CA'
b'\x00\x00\x10B'
b'f3CA'
b'\x00\x00\x10B'
b'f3CA'
b'\x00\x00\x10B'
b'f3CA'
b'\x00\x00\x10B'
b'f3CA'
b'\x00\x00\x10B'
b'f3CA'
b'\x00\x00\x10B'zycja_x: 28.0 | pozycja_y: 289.8 | kierunek: LEWO[1] | lvl: 0                 
b'f3CA'
b'\x00\x00\x10B'zycja_x: 28.0 | pozycja_y: 289.8 | kierunek: LEWO[1] | lvl: 0                 
b'f3CA'
b'\x00\x00\x10B'zycja_x: 28.0 | pozycja_y: 289.8 | kierunek: LEWO[1] | lvl: 0                 
b'f3CA'
b'\x00\x00\x10B'zycja_x: 28.0 | pozycja_y: 289.8 | kierunek: LEWO[1] | lvl: 0                 
b'f3CA'
b'\x00\x00\x10B'zycja_x: 28.0 | pozycja_y: 289.8 | kierunek: LEWO[1] | lvl: 0                 
b'f3CA'
b'\x00\x00\x10B'zycja_x: 28.0 | pozycja_y: 289.8 | kierunek: LEWO[1] | lvl: 0                 
b'f3CA'
b'\x00\x00\x10B'zycja_x: 28.0 | pozycja_y: 289.8 | kierunek: LEWO[1] | lvl: 0                 
b'f3CA'
b'\x00\x00\x10B'zycja_x: 28.0 | pozycja_y: 289.8 | kierunek: LEWO[1] | lvl: 0                 
b'f3CA'
b

In [ ]:
# --- GŁÓWNA CZĘŚĆ SKRYPTU ---



print("\n[*] Monitor wystartował. Uruchamiam logikę bota...")

# Wolniejsza pętla logiczna bota
while True:
    # Możesz teraz uzależnić akcje od aktualnego stanu z wątku w tle!
    # np. if game_state["x"] > 150.0: ...
    
    # Symulacja akcji (trwająca określony czas)
    hooker.input_move([np.random.choice(["LEFT","RIGHT","PASS"])], duration=0.5*np.random.random())
    hooker.input_move(["SPACEBAR", np.random.choice(["LEFT","RIGHT"])], duration=np.random.random())